In [254]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine, text

In [256]:
# tsega kidanu, dxr9fv
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Password2025^^"

src_dbname = "chinook"
dst_dbname = "chinook_dw2"

In [258]:
# Create a New Empty Data Warehouse database in MySQL:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
connection = sqlEngine.connect()

connection.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
connection.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
connection.execute(text(f"USE {dst_dbname};"))

connection.close()

In [260]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [262]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

Running SQL Alchemy Version: 2.0.30
Running PyMongo Version: 4.10.1


In [264]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Password2025^^",
    "hostname" : "localhost",
    "dbname" : "chinook_dw2"
}


# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "cluster_location" : "local", # "atlas"
    "db_name" : "chinook_purchasing"
}

In [266]:
def get_sql_dataframe(sql_query, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    dframe = pd.read_sql(text(sql_query), connection);
    connection.close()
    
    return dframe
    

def set_dataframe_mongo(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


def get_mongo_client(**args):
    '''Validate proper input'''
    if args["cluster_location"] not in ['atlas', 'local']:
        raise Exception("You must specify either 'atlas' or 'local' for the cluster_location parameter.")
    
    else:
        if args["cluster_location"] == "atlas":
            connect_str = f"mongodb+srv://{args['user_name']}:{args['password']}@"
            connect_str += f"{args['cluster_name']}.{args['cluster_subnet']}.mongodb.net"
            client = pymongo.MongoClient(connect_str, tlsCAFile=certifi.where())
            
        elif args["cluster_location"] == "local":
            client = pymongo.MongoClient("mongodb://localhost:27017/")
        
    return client


def get_mongo_dataframe(mongo_client, db_name, collection, query):
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = mongo_client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    mongo_client.close()
    
    return dframe


def set_mongo_collections(mongo_client, db_name, data_directory, json_files):
    db = mongo_client[db_name]
    
    for file in json_files:
        db.drop_collection(file)
        json_file = os.path.join(data_directory, json_files[file])
        with open(json_file, 'r') as openfile:
            json_object = json.load(openfile)
            file = db[file]
            result = file.insert_many(json_object)
        
    mongo_client.close()

In [269]:
#read the data from your newly created CSV file into a Pandas Dataframe
data_dir = os.path.join(os.getcwd(), 'data')
df_track = pd.read_csv(os.path.join(data_dir, 'chinook_track.csv'))
df_track.head()


,TrackId,TrackName,Composer,Milliseconds,UnitPrice,AlbumId,AlbumTitle,GenreId,GenreName,MediaTypeId,MediaTypeName
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
1,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
2,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
3,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
4,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file


In [271]:
# add new key
df_track.insert(0, "track_key", range(1, df_track.shape[0]+1))
df_track.head()


,track_key,TrackId,TrackName,Composer,Milliseconds,UnitPrice,AlbumId,AlbumTitle,GenreId,GenreName,MediaTypeId,MediaTypeName
0,1,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
1,2,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
2,3,7,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
3,4,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file
4,5,9,Snowballed,"Angus Young, Malcolm Young, Brian Johnson",203102,0.99,1,For Those About To Rock We Salute You,1,Rock,1,MPEG audio file


In [273]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [275]:
# write it back to the MySQL data warehouse database as a new dimension table make dim track

db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_track, 'dim_track', 'track_key', db_operation)

In [277]:
# load the data in your newly created JSON export file into a new MongoDB Atlas database collectio
client = get_mongo_client(**mongodb_args)

data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"customer" : 'chinook_customers.json'}

set_mongo_collections(client, mongodb_args["db_name"], data_dir, json_files)     

In [279]:
#read data from your newly created MongoDB collection into a Pandas Dataframe, make any necessary transformations, 
client = get_mongo_client(**mongodb_args)

query = {} # Select all elements (columns), and all documents (rows).
collection = "customer"

df_customer = get_mongo_dataframe(client, mongodb_args["db_name"], collection, query)
#and then write it back to the MySQL data warehouse database as a new dimension table

drop_cols = ['Email', 'PostalCode']
df_customer.drop(drop_cols, axis=1, inplace=True)
df_customer.insert(0, "customer_key", range(1, df_customer.shape[0]+1))



In [281]:
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, df_customer, 'dim_customer', 'customer_key', db_operation)

In [283]:
sql_invoice = "SELECT * FROM chinook.invoice;"
df_invoice = get_dataframe(user_id, pwd, host_name, src_dbname, sql_invoice)
df_invoice.head(2)

drop_cols = ['BillingState', 'BillingAddress', 'BillingPostalCode', 'BillingPostalCode' ]
df_invoice.drop(drop_cols, axis=1, inplace=True)
df_invoice.insert(0, "invoice_key", range(1, df_invoice.shape[0]+1))
# Using code from the Lab 4 notebook, read data from a third entity (customer, product, etc.) into a Pandas Dataframe,
# make any necessary transformations, and then write it to the MySQL data warehouse database as a new dimension table

In [285]:
df_invoice.head()

,invoice_key,InvoiceId,CustomerId,InvoiceDate,BillingCity,BillingCountry,Total
0,1,1,2,2021-01-01,Stuttgart,Germany,1.98
1,2,2,4,2021-01-02,Oslo,Norway,3.96
2,3,3,8,2021-01-03,Brussels,Belgium,5.94
3,4,4,14,2021-01-06,Edmonton,Canada,8.91
4,5,5,23,2021-01-11,Boston,USA,13.86


In [287]:
sql_invoiceLine = "SELECT * FROM chinook.invoiceLine;"
df_invoiceLine = get_dataframe(user_id, pwd, host_name, src_dbname, sql_invoiceLine)

In [289]:
df_invoiceLine.insert(0, "invoiceLine_key", range(1, df_invoiceLine.shape[0]+1))
df_invoiceLine.head(2)
# Using code from the Lab 4 notebook, read data from a third entity (customer, product, etc.) into a Pandas Dataframe,
# make any necessary transformations, and then write it to the MySQL data warehouse database as a new dimension table

,invoiceLine_key,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,1,2,0.99,1
1,2,2,1,4,0.99,1


In [292]:
sql_album = "SELECT * FROM chinook.album;"
df_album = get_dataframe(user_id, pwd, host_name, src_dbname, sql_album)
df_album.insert(0, "album_key", range(1, df_album.shape[0]+1))
df_album.head(2)

,album_key,AlbumId,Title,ArtistId
0,1,1,For Those About To Rock We Salute You,1
1,2,2,Balls to the Wall,2


In [294]:
sql_artist = "SELECT * FROM chinook.artist;"
df_artist = get_dataframe(user_id, pwd, host_name, src_dbname, sql_artist)
df_artist.insert(0, "artist_key", range(1, df_artist.shape[0]+1))
df_artist.head(2)

,artist_key,ArtistId,Name
0,1,1,AC/DC
1,2,2,Accept


In [296]:
sql_genre = "SELECT * FROM chinook.genre;"
df_genre = get_dataframe(user_id, pwd, host_name, src_dbname, sql_genre)
df_genre.insert(0, "genre_key", range(1, df_genre.shape[0]+1))
df_genre.head(2)

,genre_key,GenreId,Name
0,1,1,Rock
1,2,2,Jazz


In [298]:
sql_playlist = "SELECT * FROM chinook.playlist;"
df_playlist = get_dataframe(user_id, pwd, host_name, src_dbname, sql_playlist)
df_playlist.insert(0, "playlist_key", range(1, df_playlist.shape[0]+1))
df_playlist.head(2)

,playlist_key,PlaylistId,Name
0,1,1,Music
1,2,2,Movies


In [300]:
sql_playlistTrack = "SELECT * FROM chinook.playlistTrack;"

# Fetch the data using the get_dataframe function
df_playlistTrack = get_dataframe(user_id, pwd, host_name, src_dbname, sql_playlistTrack)
df_playlistTrack.insert(0, "playlistTrack_key", range(1, df_playlistTrack.shape[0]+1))
df_playlistTrack.head(2)

,playlistTrack_key,PlaylistId,TrackId
0,1,1,1
1,2,1,2


In [302]:
sql_employee = "SELECT * FROM chinook.employee;"

# Fetch the data using the get_dataframe function
df_employee= get_dataframe(user_id, pwd, host_name, src_dbname, sql_employee)
df_employee.insert(0, "employee_key", range(1, df_employee.shape[0]+1))
df_employee.head(2)

,employee_key,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


In [241]:
##### create factor table
## need customer id,first name, last name
## need track if id, track name, composer id, albumid (because i joined these all intro track table)
## need album id
## need genre id
# tran date (maybe invoice date im not sure)


In [306]:
# fetch from dim 
sql_dim_date = "SELECT date_key, full_date FROM chinook_dw2.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(2)

,date_key,full_date
0,20000101,2000-01-01
1,20000102,2000-01-02


In [308]:
# repalce invoice date
df_dim_invoice_date = df_dim_date.rename(columns={"date_key" : "invoice_date_key", "full_date" : "InvoiceDate"})
df_invoice.InvoiceDate = df_invoice.InvoiceDate.astype('datetime64[ns]').dt.date
df_invoice = pd.merge(df_invoice, df_dim_invoice_date, on='InvoiceDate', how='left')
df_invoice.drop(['InvoiceDate'], axis=1, inplace=True)
df_invoice.head(2)

,invoice_key,InvoiceId,CustomerId,BillingCity,BillingCountry,Total,invoice_date_key
0,1,1,2,Stuttgart,Germany,1.98,20210101.0
1,2,2,4,Oslo,Norway,3.96,20210102.0


In [312]:
db_operation = "insert"

tables = [
    ('dim_invoiceline', df_invoiceLine, 'invoiceLine_key'),
    ('dim_invoice', df_invoice, 'invoice_key'),
    ('dim_employee', df_employee, 'employee_key'),
    ('dim_artist', df_artist, 'artist_key'),
    ('dim_album', df_album, 'album_key'),
    ('dim_genre', df_genre, 'genre_key'),
    ('dim_track', df_track, 'track_key'),
    ('dim_playlist', df_playlist, 'playlist_key'),
    ('dim_playlisttrack', df_playlistTrack, 'playlistTrack_key')  
]

for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [316]:
# import dim_invoiceline from dst
# import dim_invoice from dst
# import dim_track from dst
# import dim_customer from dst

sql_dim_customers = "SELECT * FROM chinook_dw2.dim_customer;"
df_dim_customer = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_customers)

sql_dim_track = "SELECT * FROM chinook_dw2.dim_track;"
df_dim_track = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_track)

sql_dim_invoice = "SELECT * FROM chinook_dw2.dim_invoice;"
df_dim_invoice = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_invoice)

sql_dim_invoiceline = "SELECT * FROM chinook_dw2.dim_invoiceline"
df_dim_invoiceline = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_dim_invoiceline)


In [326]:
import pandas as pd

# merge and make fact table
df_fact_sales = pd.merge(df_dim_customer, df_dim_invoice, on='CustomerId', how='inner')
df_fact_sales = pd.merge(df_fact_sales, df_dim_invoiceline, on='InvoiceId', how='inner')
df_fact_sales = pd.merge(df_fact_sales, df_dim_track, on='TrackId', how='inner')



df_fact_sales.rename(columns={'UnitPrice_y': 'UnitPrice'}, inplace=True)

drop_cols = ['MediaTypeId', 'MediaTypeName', 'InvoiceId', 'InvoiceLineId', 'CustomerId', 'TrackId'] 
df_sales.drop(drop_cols, axis=1, inplace=True)

df_fact_sales.insert(0, "sale_key", range(1, df_sales.shape[0] + 1))


df_fact_sales.columns
# drop InvoiceId, InvoiceLineId, CustomerId TrackId

Index(['sale_key', 'customer_key', 'CustomerId', 'FirstName', 'LastName',
       'SupportRepId', 'Country', 'invoice_key', 'InvoiceId', 'BillingCity',
       'BillingCountry', 'Total', 'invoice_date_key', 'invoiceLine_key',
       'InvoiceLineId', 'TrackId', 'UnitPrice_x', 'Quantity', 'track_key',
       'TrackName', 'Composer', 'Milliseconds', 'UnitPrice', 'AlbumId',
       'AlbumTitle', 'GenreId', 'GenreName', 'MediaTypeId', 'MediaTypeName'],
      dtype='object')

In [328]:
df_fact_sales.head()


,sale_key,customer_key,CustomerId,FirstName,LastName,SupportRepId,Country,invoice_key,InvoiceId,BillingCity,...,TrackName,Composer,Milliseconds,UnitPrice,AlbumId,AlbumTitle,GenreId,GenreName,MediaTypeId,MediaTypeName
0,1,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Shout It Out Loud,"Paul Stanley, Gene Simmons, B. Ezrin",219742,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
1,2,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Calling Dr. Love,Gene Simmons,225332,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
2,3,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Strutter,"Paul Stanley, Gene Simmons",192496,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
3,4,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Cold Gin,Ace Frehley,262243,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
4,5,1,1,LuÃ­s,GonÃ§alves,3,Brazil,327,327,São José dos Campos,...,Interlude Zumbi,Chico Science,71627,0.99,24,Afrociberdelia,7,Latin,1,MPEG audio file


In [330]:
db_operation = "insert"

# write back the table name and primary keys
tables = [
    ('fact_purchase_orders', df_fact_sales, 'sale_key'),
]

for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)


In [332]:
#Validate the correctness of the new "sales Orders"te_purchase_orders = "SELECT * FROM chinook_dw2.fact_purchase_orders;"

df_count_purchase_orders = get_sql_dataframe(sql_validate_purchase_orders, **mysql_args)

df_count_purchase_orders.head()

,sale_key,customer_key,CustomerId,FirstName,LastName,SupportRepId,Country,invoice_key,InvoiceId,BillingCity,...,TrackName,Composer,Milliseconds,UnitPrice,AlbumId,AlbumTitle,GenreId,GenreName,MediaTypeId,MediaTypeName
0,1,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Shout It Out Loud,"Paul Stanley, Gene Simmons, B. Ezrin",219742,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
1,2,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Calling Dr. Love,Gene Simmons,225332,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
2,3,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Strutter,"Paul Stanley, Gene Simmons",192496,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
3,4,1,1,LuÃ­s,GonÃ§alves,3,Brazil,121,121,São José dos Campos,...,Cold Gin,Ace Frehley,262243,0.99,37,Greatest Kiss,1,Rock,1,MPEG audio file
4,5,1,1,LuÃ­s,GonÃ§alves,3,Brazil,327,327,São José dos Campos,...,Interlude Zumbi,Chico Science,71627,0.99,24,Afrociberdelia,7,Latin,1,MPEG audio file


In [334]:
sql_most_bought_track = """
    SELECT 
        TrackName, 
        SUM(Quantity) AS TotalQuantitySold
    FROM 
        fact_purchase_orders
    GROUP BY 
        TrackName
    ORDER BY 
        TotalQuantitySold DESC
    LIMIT 1;
"""

In [336]:
# df_fact_inventory_transactions = get_sql_dataframe(sql_most_bought_track, **mysql_args)
# df_fact_inventory_transactions

,TrackName,TotalQuantitySold
0,Linha Do Equador,3.0


In [360]:
# sql_employee_most_sales="""SELECT 
#     e.FirstName, e.LastName
#     SUM(f.Quantity * f.UnitPrice_x) AS TotalSales
# FROM 
#     fact_purchase_orders f
# JOIN 
#     dim_employee e ON f.SupportRepId = e.EmployeeId
# GROUP BY 
#     e.EmployeeName
# ORDER BY 
#     TotalSales DESC
# LIMIT 1;
# """


In [400]:
sql_employee_most_sales="""SELECT 
    e.EmployeeId, 
    e.FirstName, 
    e.LastName, 
    SUM(f.Quantity * f.UnitPrice_x) AS TotalSales
FROM 
    fact_purchase_orders f
JOIN 
    dim_employee e ON f.SupportRepId = e.EmployeeId
GROUP BY 
    e.EmployeeId, e.FirstName, e.LastName 
ORDER BY 
    TotalSales DESC
"""


In [402]:
df_fact_inventory_transactions = get_sql_dataframe(sql_employee_most_sales, **mysql_args)
df_fact_inventory_transactions

,EmployeeId,FirstName,LastName,TotalSales
0,3,Jane,Peacock,265.32
1,4,Margaret,Park,208.89
2,5,Steve,Johnson,175.23


In [448]:
sql_sales_by_month="""SELECT 
    d.fiscal_year,  -- Add fiscal_year for clarity
    d.fiscal_month_of_year AS month
    SUM(f.Total) AS total_sales, 
    COUNT(f.invoice_key) AS total_orders  
FROM 
    fact_purchase_orders f
JOIN 
    dim_date d ON f.invoice_date_key = d.date_key 
GROUP BY 
    d.fiscal_year, d.fiscal_month_of_year 
ORDER BY 
    d.fiscal_year DESC, d.fiscal_month_of_year DESC;  

"""

In [450]:
df_fact_inventory_transactions = get_sql_dataframe(sql_sales_by_month, **mysql_args)
df_fact_inventory_transactions

,fiscal_year,month,total_sales,total_orders
0,2024,6,41.58,3
1,2024,5,140.58,24
2,2024,4,334.62,38
3,2024,3,334.62,38
4,2024,2,334.62,38
5,2024,1,256.41,21
6,2023,2,153.45,24
7,2023,1,334.62,38
8,2022,12,334.62,38
9,2022,11,334.62,38
